# 1.Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics as mt
from sklearn import linear_model as lm
import pickle

# 2. Funcao para carregar os datasets

In [2]:
def Load_Data_Set(df_x_train, df_y_train):
    df1 = pd.read_csv(df_x_train)
    df2 = pd.read_csv(df_y_train)
    return df1, df2

# 3. Criando e concatenando os dataframes

### 3.1 Dataset de treinamento

In [3]:
df_x_train = "../2_ensaio_regressao/1_dados_treinamento/X_training.csv"

In [4]:
df_y_train = "../2_ensaio_regressao/1_dados_treinamento/y_training.csv"

In [5]:
df1_train, df2_train = Load_Data_Set(df_x_train, df_y_train)

In [6]:
df1_train['label'] = df2_train

In [7]:
df_train = df1_train.copy()

### 3.2 Dataset de validacao

In [8]:
df_x_val = "../2_ensaio_regressao/2_dados_validacao/X_validation.csv"

In [9]:
df_y_val = "../2_ensaio_regressao/2_dados_validacao/y_val.csv"

In [10]:
df1_val, df2_val = Load_Data_Set(df_x_val, df_y_val)

In [11]:
df1_val['label'] = df2_val

In [12]:
df_val = df1_val.copy()

### 3.3 Dataset de teste

In [13]:
df_x_test = "../2_ensaio_regressao/3_dados_teste/X_test.csv"

In [14]:
df_y_test = "../2_ensaio_regressao/3_dados_teste/y_test.csv"

In [15]:
df1_test, df2_test = Load_Data_Set(df_x_test, df_y_test)

In [16]:
df1_test['label'] = df2_test

In [17]:
df_test = df1_test.copy()

# 4. Funções para treinar o modelo e avaliar as métricas

In [18]:
#Criando uma função para Treinar o Modelo encima dos dados de treinamento

#Fazer as previsoes encima dos proprios dados de treinamento

#Entrada: dataframe de treinamento + parametros de ajuste do modelo, a saber alpha e max_iter

#Saida: valores de y_train + valores de previsao feitos a partir dos dados de treinamento (y_pred_train) +
# algoritmo lasso treinado

def Model_Training(df_train, alpha, max_iter):
    
    features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
    label = ['label']
    
    #Preparação dos dados de treino
    x_train = df_train.loc[:, features]

    #Esse metodo ravel transforma os valores em um array
    y_train = df_train.loc[:, label].values.ravel()      
    
    #Treinamento do algoritmo Lasso
      
    lasso = lm.Lasso(alpha = alpha, max_iter = max_iter)    
  
    lasso.fit(x_train, y_train)

    #Fazendo previsoes emcima dos dados de treinamento
    y_pred_train = lasso.predict(x_train)
    
    return y_train, y_pred_train, lasso
   


In [19]:
def Previsoes_Dataframe_Test(df_test, lasso):
    
    #Entrada: dataset de teste +  algoritmo lasso treinado
    
    #Saida: os dados de y do proprio dataset de teste (y_test) e as previsoes feitas encima dos dados de teste (y_pred_test)
    
    features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
    label = ['label'] 
       
    #Separando os dados
    x_test = df_test.loc[:, features]

    y_test = df_test.loc[:, label].values.ravel() 
    
    #Fazendo previsões
    y_pred_test = lasso.predict(x_test)
    
    return y_test, y_pred_test 
       

In [20]:
def Previsoes_Dataframe_Val(df_val, lasso):
    
   #Entrada: dataset de validacao +  algoritmo laso treinado
    
   #Saida: os dados de y do proprio dataset de validacao (y_val) e as previsoes feitas encima dos dados de validacao
   #(y_pred_val)
    

    features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
    label = ['label'] 
    
           
    #Separando os dados
    x_val = df_val.loc[:, features]

    y_val = df_val.loc[:, label].values.ravel() 
    
    #Fazendo previsões
    y_pred_val = lasso.predict(x_val)
    
    return y_val, y_pred_val

In [21]:
#Criando uma função para avaliar as metricas do modelo

#Entrada: Os proprios valores da label(y) do dataset que precisamos avaliar as metricas,
# as previsoes feitas a partir desses dados, parametros de ajuste do modelo, a saber alpha e max_iter

#Saida: Dataframe com as principais metricas do modelo

def Model_Metrics(y, y_pred, alpha, max_iter):
    
    #R2_Score
    r2_score = np.round(mt.r2_score(y , y_pred ),4)
        
    #Mean_Squared_Error (MSE)
    mean_squared_error = np.round(mt.mean_squared_error(y, y_pred ),4)
        
    #Root Mean_Squared Error (RMSE)
    rmse = np.round(np.sqrt(mean_squared_error),4)
        
    #Mean_Absolute_Error(MAE)
    mean_absolute_error = np.round(mt.mean_absolute_error (y,y_pred),4)
   
    #Mean Absolute Percentage Error (MAPE)
    mean_absolute_percentage_error = np.round(mt.mean_absolute_percentage_error (y,y_pred),4)
    
    #Criando um dataframe com as metricas
    
    
    parameters = 'alpha = ' + str(alpha) + ' ;max_iter = ' + str(max_iter) 
    
    d = { parameters: [r2_score, mean_squared_error , rmse , mean_absolute_error ,mean_absolute_percentage_error]}  
    
    df_metrics =  pd.DataFrame(data = d, index = ['r2_score', 'mse', 'rmse', 'mae',
                                                 'mape'])      
    
        
    return df_metrics

# 5. Treinamento do modelo, validação e verificação de performance

### 5.1 Treinar o modelo com os valores de parametro default: alpha = 1, max_iter = 1000 e verificação das métricas do modelo sobre os dados de treinamento

In [22]:
y_train, y_pred_train, lasso = Model_Training(df_train, 1, 1000)
df_default_train = Model_Metrics(y_train, y_pred_train, 1, 1000)
df_default_train

,alpha = 1 ;max_iter = 1000
r2_score,0.0074
mse,474.4748
rmse,21.7824
mae,17.3055
mape,8.7367


### 5.2 Testar o algoritmo com o parametro default  nos dados de validação e verificar sua performance

In [23]:
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val, lasso)
df_default_val = Model_Metrics(y_val, y_pred_val, 1, 1000)
df_default_val

,alpha = 1 ;max_iter = 1000
r2_score,0.0079
mse,473.7471
rmse,21.7657
mae,17.2649
mape,8.6958


### 5.3 Treinar o modelo nos dados de treinamento alterando os parametros e testar a performance desse modelo sobre os dados de validação

### a) alpha = 5, max_iter = 1000

In [24]:
y_train, y_pred_train, lasso = Model_Training(df_train, 5, 1000)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val,lasso) 
df_val1 = Model_Metrics(y_val, y_pred_val, 5,1000)
df_val1

,alpha = 5 ;max_iter = 1000
r2_score,-0.0000
mse,477.5120
rmse,21.8520
mae,17.3528
mape,8.6787


### b) alpha = 100, max_iter = 1000

In [25]:
y_train, y_pred_train, lasso = Model_Training(df_train, 100, 1000)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val,lasso) 
df_val2 = Model_Metrics(y_val, y_pred_val, 100,1000)
df_val2

,alpha = 100 ;max_iter = 1000
r2_score,-0.0000
mse,477.5120
rmse,21.8520
mae,17.3528
mape,8.6787


### c) alpha = 100, max_iter = 2000

In [26]:
y_train, y_pred_train, lasso = Model_Training(df_train, 100, 2000)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val,lasso) 
df_val3 = Model_Metrics(y_val, y_pred_val, 100,2000)
df_val3

,alpha = 100 ;max_iter = 2000
r2_score,-0.0000
mse,477.5120
rmse,21.8520
mae,17.3528
mape,8.6787


### d) alpha = 500, max_iter = 100

In [27]:
y_train, y_pred_train, lasso = Model_Training(df_train, 500, 1000)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val,lasso) 
df_val4 = Model_Metrics(y_val, y_pred_val, 500,100)
df_val4

,alpha = 500 ;max_iter = 100
r2_score,-0.0000
mse,477.5120
rmse,21.8520
mae,17.3528
mape,8.6787


### e) alpha = 3, max_iter = 20

In [28]:
y_train, y_pred_train, lasso = Model_Training(df_train, 3, 20)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val,lasso) 
df_val5 = Model_Metrics(y_val, y_pred_val, 3,20)
df_val5

,alpha = 3 ;max_iter = 20
r2_score,-0.0000
mse,477.5120
rmse,21.8520
mae,17.3528
mape,8.6787


### f) alpha = 1, max_iter = 20

In [29]:
y_train, y_pred_train, lasso = Model_Training(df_train, 1, 20)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val,lasso) 
df_val6= Model_Metrics(y_val, y_pred_val, 1,20)
df_val6

,alpha = 1 ;max_iter = 20
r2_score,0.0079
mse,473.7471
rmse,21.7657
mae,17.2649
mape,8.6958


### g) alpha = 0.4, max_iter = 20

In [30]:
y_train, y_pred_train, lasso = Model_Training(df_train, 0.4, 20)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val,lasso) 
df_val7 = Model_Metrics(y_val, y_pred_val, 0.4,20)
df_val7

,alpha = 0.4 ;max_iter = 20
r2_score,0.0133
mse,471.1832
rmse,21.7068
mae,17.2126
mape,8.6893


### h) alpha = 0.2, max_iter = 15

In [31]:
y_train, y_pred_train, lasso = Model_Training(df_train, 0.2, 15)
y_val, y_pred_val = Previsoes_Dataframe_Val(df_val,lasso) 
df_val8 = Model_Metrics(y_val, y_pred_val, 0.2,15)
df_val8

,alpha = 0.2 ;max_iter = 15
r2_score,0.0296
mse,463.3853
rmse,21.5264
mae,17.0933
mape,8.6773


### 5.4 Verificar o parametro que obteve os melhores resultados nos dados de validação

In [32]:
df_metrics_val = pd.concat([df_default_val, df_val1, df_val2, df_val3, df_val4, df_val5,
                           df_val6, df_val7, df_val8], axis = 1)
df_metrics_val

,alpha = 1 ;max_iter = 1000,alpha = 5 ;max_iter = 1000,alpha = 100 ;max_iter = 1000,alpha = 100 ;max_iter = 2000,alpha = 500 ;max_iter = 100,alpha = 3 ;max_iter = 20,alpha = 1 ;max_iter = 20,alpha = 0.4 ;max_iter = 20,alpha = 0.2 ;max_iter = 15
r2_score,0.0079,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0079,0.0133,0.0296
mse,473.7471,477.5120,477.5120,477.5120,477.5120,477.5120,473.7471,471.1832,463.3853
rmse,21.7657,21.8520,21.8520,21.8520,21.8520,21.8520,21.7657,21.7068,21.5264
mae,17.2649,17.3528,17.3528,17.3528,17.3528,17.3528,17.2649,17.2126,17.0933
mape,8.6958,8.6787,8.6787,8.6787,8.6787,8.6787,8.6958,8.6893,8.6773


#### Observação: A dupla de parâmetros que encontramos a melhor performance foi - alpha = 0.2, max_iter = 15, que é o objeto df_val8

# 6. Unificar os dados de treinamento e validação e retreinar o algoritmo, utilizando os melhores valores para os parâmetros

### 6.1 Dividir novamente os datasets de treinamento e validação

In [33]:
features = ['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode',
       'speechiness', 'tempo', 'time_signature', 'audio_valence']
    
    
label = ['label'] 

#Separação dos dados de treino
x_train = df_train.loc[:, features]

#Esse metodo ravel transforma os valores em um array
y_train = df_train.loc[:, label].values.ravel()      

#Separação dos dados de validação
x_val = df_val.loc[:, features]

y_val = df_val.loc[:, label].values.ravel() 

### 6.2 Treinar o modelo novamente com o melhor parametro obtido

In [34]:
model_last = lm.Lasso(alpha = 0.2, max_iter = 15)
model_last.fit( np.concatenate( (x_train, x_val) ), np.concatenate((y_train, y_val)) )

Lasso(alpha=0.2, max_iter=15)

### 6.3 Fazendo previsões nos dados de teste 

In [35]:
y_test, y_pred_test = Previsoes_Dataframe_Test(df_test, model_last)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


### 6.4 Verificando as métricas do modelo com os dados de teste

In [36]:
df_metrics_test = Model_Metrics(y_test, y_pred_test, 0.2, 15)
df_metrics_test

,alpha = 0.2 ;max_iter = 15
r2_score,0.0332
mse,470.7580
rmse,21.6970
mae,17.2587
mape,8.6533


# 7. Resguardando os objetos com as métricas de maior performance

### 7.1 Alterando o nome das colunas dos dataframes

#### a) dataframe teste com melhores métricas

In [37]:
df_metrics_test = df_metrics_test.rename(columns = {"alpha = 0.2 ;max_iter = 15": "Linear Regression Lasso"})
df_metrics_test

,Linear Regression Lasso
r2_score,0.0332
mse,470.7580
rmse,21.6970
mae,17.2587
mape,8.6533


#### b) dataframe validação com melhores métricas

In [38]:
df_val8 = df_val8.rename(columns = {"alpha = 0.2 ;max_iter = 15": "Linear Regression Lasso"})
df_val8

,Linear Regression Lasso
r2_score,0.0296
mse,463.3853
rmse,21.5264
mae,17.0933
mape,8.6773


#### c) dataframe treinamento com melhores métricas

In [39]:
df_default_train = df_default_train.rename(columns={"alpha = 1 ;max_iter = 1000": "Linear Regression Lasso"})
df_default_train

,Linear Regression Lasso
r2_score,0.0074
mse,474.4748
rmse,21.7824
mae,17.3055
mape,8.7367


# 8. Salvar os objetos com as melhores metricas em um arquivo pickle

### a) dataframe com métricas dos dados de treinamento

In [40]:
with open('arquivo_lasso_train.pkl', 'wb') as arquivo_lasso_train:
    pickle.dump(df_default_train,arquivo_lasso_train)  

In [41]:
arquivo_lasso_train.close()

### b) dataframe com métricas dos dados de validação

In [42]:
with open('arquivo_lasso_val.pkl', 'wb') as arquivo_lasso_val:
    pickle.dump(df_val8,arquivo_lasso_val)  

In [43]:
arquivo_lasso_val.close()

### c) dataframe com métricas dos dados de teste

In [44]:
with open('arquivo_lasso_test.pkl', 'wb') as arquivo_lasso_test:
    pickle.dump(df_metrics_test,arquivo_lasso_test)  

In [45]:
arquivo_lasso_test.close()